# Info

현 예측은 아무런 EDA 과정 없이 추측만을 가지고 Feature를 선택하고 진행하며 모델 선택 및 적용 과정에서도 미흡한 부분이 다수 있을 가능성이 높음.

---------------------------------------
본 파일은 로스트아크의 캐릭터 스펙(Feature: 각인, 보석, 카드 등)에 따른 아이템 레벨을 예측하고 

현재 아이템 레벨에 비해 스펙이 낮을 경우 스펙을 높일 것을 추천하고 

아이템 레벨에 비해 스펙이 높을 경우 아이템 레벨을 높일 것을 추천하는 것을 목표로 함.

---------------------------------------
더 나아가 어떤 스펙을 어느 정도로 올리면 좋은지도 추천해주고자 함.

1. profile_table: expeditionLevel, totalSkillPoint, characterLevel, itemMaxLevel
2. accessory_table: accessory_quality, accessory_grade, accessory_tier, 팔찌_효과, AS_setLevel
3. avatar_table: avatar_grade
4. card_table: setName, setPoint
5. engraving_table: grantName, grantPoint, engraving_name, engraving_level
6. equipment_table: equipment_quality, equipment_grade, equipment_setLevel
7. gem_table: gem_grade, gem_level, gem_tier
8. skill_table: skill_tripod_point
9. stats_table: stat_값

In [1]:
import lostark as lok
import pandas as pd
import numpy as np
import joblib
from pprint import pprint

In [2]:
### DataBase ###
db, cursor = lok.get_db_cursor()

In [11]:
tables = ['profile_table','accessory_table', 'avatar_table', 'card_table', 'engraving_table', 
         'equipment_table', 'gem_table', 'skill_table', 'stats_table']
accessories = ['목걸이', '귀걸이1', '귀걸이2', '반지1', '반지2']
accessory_column = ['_quality', '_grade', '_tier']
avatars = ['무기1', '무기2', '상의1', '상의2', '하의1', '하의2']
avatar_column = ['_grade']
equipments = ['무기', '투구', '어깨', '상의', '하의', '장갑']
equipment_column = ['_quality', '_grade', '_setLevel']
gem_column = ['_grade', '_level', '_tier']
stat_column = ['치명', '특화', '신속', '제압', '인내', '숙련']

sql = f"""SELECT profile_table.characterCode, profile_table.expeditionLevel, profile_table.totalSkillPoint,
profile_table.characterLevel, profile_table.itemMaxLevel,

{', '.join(f"accessory_table.{accessory}{column}" for accessory in accessories for column in accessory_column)},

{', '.join(f"avatar_table.{avatar}{column}" for avatar in avatars for column in avatar_column)},

{', '.join(f"card_table.setName{i}, card_table.setPoint{i}" for i in range(1, 5))},

{', '.join(f"engraving_table.grantName{i}, engraving_table.grantPoint{i}" for i in range(1, 3))},
{', '.join(f"engraving_table.engraving{i}_name, engraving_table.engraving{i}_level" for i in range(1, 12))},

{', '.join(f"equipment_table.{equipment}{column}" for equipment in equipments for column in equipment_column)},

{', '.join(f"gem_table.gem{i}{column}" for i in range(1, 12) for column in gem_column)},

{', '.join(f"skill_table.skill{i}_tripod{j}_point" for i in range(1, 17) for j in range(1, 4))},

{', '.join(f"stats_table.{column}_값" for column in stat_column)}

FROM profile_table
{' '.join(f"INNER JOIN {tables[i+1]} ON {tables[i]}.characterCode = {tables[i+1]}.characterCode"
for i in range(len(tables)-1))};
"""
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall())

In [12]:
columns = ["characterCode", "expeditionLevel", "totalSkillPoint", "characterLevel", "itemMaxLevel"]
for accessory in accessories:
    for column in accessory_column:
        columns.append(f"{accessory}{column}")
for avatar in avatars :
    for column in avatar_column:
        columns.append(f"avatar_{avatar}{column}") 
for i in range(1, 5):
    columns.append(f"card_setName{i}")
    columns.append(f"card_setPoint{i}")
for i in range(1, 3):
    columns.append(f"engraving_grantName{i}")
    columns.append(f"engraving_grantPoint{i}")
for i in range(1, 12):
    columns.append(f"engraving{i}_name")
    columns.append(f"engraving{i}_level")
for equipment in equipments:
    for column in equipment_column:
        columns.append(f"equipment_{equipment}{column}")
for i in range(1, 12):
    for column in gem_column:
        columns.append(f"gem{i}{column}")
for i in range(1, 17):
    for j in range(1, 4):
        columns.append(f"skill{i}_tripod{j}_point")
for column in stat_column:
    columns.append(f"{column}_값")

df.columns = columns

In [13]:
# 결측치 처리 및 데이터타입 수정
df['itemMaxLevel'] = df['itemMaxLevel'].astype('float64')
for accessory in accessories:
    df[f'{accessory}_quality'] = df[f'{accessory}_quality'].fillna(-1).astype('int64')
    df[f'{accessory}_grade'] = df[f'{accessory}_grade'].fillna('None')
    df[f'{accessory}_tier'] = df[f'{accessory}_tier'].fillna(-1).astype('int64')
for avatar in avatars :
    df[f"avatar_{avatar}_grade"] =  df[f"avatar_{avatar}_grade"].fillna('None')
for i in range(1, 5):
    df[f"card_setName{i}"] = df[f"card_setName{i}"].fillna('None')
    df[f"card_setPoint{i}"] = df[f"card_setPoint{i}"].fillna(-1).astype('int64')
for i in range(1, 3):
    df[f"engraving_grantName{i}"] = df[f"engraving_grantName{i}"].fillna('None')
    df[f"engraving_grantPoint{i}"] = df[f"engraving_grantPoint{i}"].fillna('-1').astype('int64')
for i in range(1, 12):
    df[f"engraving{i}_name"] = df[f"engraving{i}_name"].fillna('None')
    df[f"engraving{i}_level"] = df[f"engraving{i}_level"].fillna(-1).astype('int64')
for equipment in equipments:
    df[f"equipment_{equipment}_grade"] = df[f"equipment_{equipment}_grade"].fillna('None')
    df[f"equipment_{equipment}_setLevel"] = df[f"equipment_{equipment}_setLevel"].fillna(-1).astype('int64')
    df[f"equipment_{equipment}_quality"] = df[f"equipment_{equipment}_quality"].fillna(-1).astype('int64')
for i in range(1, 12):
    df[f"gem{i}_grade"] =df[f"gem{i}_grade"].fillna('None')
    df[f"gem{i}_level"] =df[f"gem{i}_level"].fillna(-1).astype('int64')
    df[f"gem{i}_tier"] =df[f"gem{i}_tier"].fillna(-1).astype('int64')
for i in range(1, 17):
    for j in range(1, 4):
        df[f"skill{i}_tripod{j}_point"] = df[f"skill{i}_tripod{j}_point"].fillna(-1).astype('int64')

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
x = df.drop(columns=['itemMaxLevel'])
x_dummies= pd.get_dummies(x)
y = df['itemMaxLevel'].astype('float64')

xtr, xt, ytr, yt = train_test_split(x_dummies, y, train_size=0.7, random_state=42)

## 학습에 시간이 오래 걸림
# rf = RandomForestRegressor(random_state=23)
# rf.fit(xtr, ytr)

# joblib.dump(rf, 'rf.pkl')

In [6]:
rf = joblib.load('rf.pkl')
rf

RandomForestRegressor(random_state=23)

In [15]:
predict_train = rf.predict(xtr)
print(mean_squared_error(ytr, predict_train))
print(mean_absolute_error(ytr, predict_train))
print(r2_score(ytr, predict_train))

29.439968052813363
3.7817139803032727
0.9941364333803223


In [16]:
predict_train = rf.predict(xt)
print(mean_squared_error(yt, predict_train))
print(mean_absolute_error(yt, predict_train))
print(r2_score(yt, predict_train))

213.23958285869256
10.201663666472133
0.9574419409122212


In [5]:
profile_responses = lok.get_total_profile_responses('roxy0')
if profile_responses != None:
    lok.insert_character_data('roxy0')

In [10]:
cursor.execute("""SELECT ArmoryProfile FROM lostark.raw_character_data_table WHERE characterName ='Sylphiette'""")
cursor.fetchone()

("{'CharacterImage': 'https://img.lostark.co.kr/armory/9/88f36a9f9a7f9c203e3604ae280c6e85f0ee6ffeb1806747197b1f39f52755c6.png?v=20230813115919', 'ExpeditionLevel': 240, 'PvpGradeName': '19급', 'TownLevel': 70, 'TownName': '샤리아', 'Title': '사랑에 빠진', 'GuildMemberGrade': '일반 길드원', 'GuildName': 'Rudius', 'UsingSkillPoint': 414, 'TotalSkillPoint': 414, 'Stats': [{'Type': '치명', 'Value': '1503', 'Tooltip': ['치명타 적중률이 53.79%증가합니다.', '물약 및 원정대 레벨 보상 효과로 24만큼 영구적으로 증가되었습니다.', '카드 도감 누적 효과가 반영된 값으로 전투정보실에서는 별도 수치를 표기하지 않습니다.', '카드 도감 누적 효과가 반영된 값으로 전투정보실에서는 별도 수치를 표기하지 않습니다.']}, {'Type': '특화', 'Value': '43', 'Tooltip': ['마력 강화 및 마력 해방의 속성 피해 효율이 13.22%증가합니다.', '신비한 마력 게이지 획득량이 1.53%증가합니다.', '각성 스킬의 피해량이 2.34%증가합니다.', '물약 및 원정대 레벨 보상 효과로 24만큼 영구적으로 증가되었습니다.', '카드 도감 누적 효과가 반영된 값으로 전투정보실에서는 별도 수치를 표기하지 않습니다.']}, {'Type': '제압', 'Value': '37', 'Tooltip': ['피격이상 및 상태이상 대상에게 주는 피해량이 2.26%증가합니다.', '무력화 대상에게 주는 피해량이 증가합니다.', '물약 및 원정대 레벨 보상 효과로 32만큼 영구적으로 증가되었습니다.', '카드 도감 누적 효과가 반영된 값으로 전투정보실에서는 별도 수치를 표기